In [51]:
import pandas as pd
import numpy as np
import torch

In [52]:
df=pd.read_csv(r'C:\Users\seven\Downloads\digit.csv')
A = df.to_numpy()
A.shape

(42000, 785)

In [53]:
Y = A[:,0]
X = A[:,1:]
X.shape

(42000, 784)

In [54]:
X=np.reshape(X,(-1,28,28,3))
X = X.astype(np.float32)
X = X.transpose((0,3,1,2))
X.shape

(14000, 3, 28, 28)

In [55]:
from torch.utils.data import DataLoader

class my_dataset(torch.utils.data.Dataset):
    def __init__(self, ):
        pass

    def __getitem__(self, index):
        return X[index],Y[index]

    def __len__(self):
        return len(X)

ds=my_dataset()

In [56]:
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data import DataLoader
# 划分训练和测试的数据
train, test = random_split(ds, (13000, 1000) )
# 设定每次读取的数量
data_loader = torch.utils.data.DataLoader(dataset=train,batch_size=1000, shuffle=True)
data_loader2 = torch.utils.data.DataLoader(dataset=test,batch_size=1000, shuffle=True)

In [57]:
import torch.nn as nn

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.L1 = nn.Conv2d(3,32,5,2,1)
        self.B1= nn.BatchNorm2d(32)

        self.L2 = nn.Conv2d(32,64,2,2,1)
        self.B2= nn.BatchNorm2d(64)
    
        
        self.L = nn.Linear(3136,10)

    def forward(self, x):
        #降二维，从start_dim开始相乘
        # x = torch.flatten(x,start_dim = 1)
        g1 = self.L1(x)
        g1 = self.B1(g1)
        # f1=torch.sigmoid(g1)
        f1=torch.relu(g1)

        g2 = self.L2(f1)
        g2 = self.B2(g2)
        f2 = torch.relu(g2)


        f2 = torch.flatten(f2,start_dim = 1)

        g = self.L(f2)

        return g
    
model = MyNet()
# model = torch.load('model5.pth')

In [58]:
import torchmetrics

lossf = nn.CrossEntropyLoss()     #loss函数
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  #SGD函数
metricsf = torchmetrics.Accuracy(task='multiclass', num_classes=10)       #torchmetrics函数

for i in range(20):
    for batchX, batchY in data_loader:
        score = model(batchX)
        score = torch.squeeze(score)
        loss = lossf(score, batchY)

        loss.backward() #求导
        optimizer.step() #梯度下降
        optimizer.zero_grad() #梯度清零

        mse = metricsf(score, batchY)    
    print(loss, metricsf.compute())
    metricsf.reset()

tensor(2.3338, grad_fn=<NllLossBackward0>) tensor(0.0995)
tensor(2.3337, grad_fn=<NllLossBackward0>) tensor(0.1047)
tensor(2.3064, grad_fn=<NllLossBackward0>) tensor(0.1089)
tensor(2.3036, grad_fn=<NllLossBackward0>) tensor(0.1129)
tensor(2.3036, grad_fn=<NllLossBackward0>) tensor(0.1229)
tensor(2.3082, grad_fn=<NllLossBackward0>) tensor(0.1291)
tensor(2.2990, grad_fn=<NllLossBackward0>) tensor(0.1308)
tensor(2.2815, grad_fn=<NllLossBackward0>) tensor(0.1356)
tensor(2.2685, grad_fn=<NllLossBackward0>) tensor(0.1418)
tensor(2.2678, grad_fn=<NllLossBackward0>) tensor(0.1449)
tensor(2.2495, grad_fn=<NllLossBackward0>) tensor(0.1505)
tensor(2.2749, grad_fn=<NllLossBackward0>) tensor(0.1536)
tensor(2.2601, grad_fn=<NllLossBackward0>) tensor(0.1558)
tensor(2.2559, grad_fn=<NllLossBackward0>) tensor(0.1597)
tensor(2.2510, grad_fn=<NllLossBackward0>) tensor(0.1634)
tensor(2.2423, grad_fn=<NllLossBackward0>) tensor(0.1681)
tensor(2.2378, grad_fn=<NllLossBackward0>) tensor(0.1690)
tensor(2.2271,

In [59]:
torch.save(model, 'model5.pth')

In [60]:
model.eval()

for batchX, batchY in data_loader2:
    score = model(batchX)
    score = torch.squeeze(score)
    mse = metricsf(score, batchY)  
      
print(metricsf.compute())

tensor(0.1050)
